# 🧠 Fine-tune T5/BART for Intent Detection + Response Generation
This notebook shows how to fine-tune a pretrained T5 or BART model on a custom dataset that includes user queries (instructions), intents, and responses.

In [2]:
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
import evaluate
import numpy as np

# Load dataset
dataset = load_dataset('csv', data_files='./nlp_intent_response_dataset.csv')
dataset = dataset['train'].train_test_split(test_size=0.1)

# Choose model: 't5-small' or 'facebook/bart-base'
model_checkpoint = 't5-small'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\huggingface_hub\file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
# Preprocessing function
max_input_length = 128
max_target_length = 128

def preprocess(example):
    inputs = tokenizer(example['input_text'], truncation=True, padding='max_length', max_length=max_input_length)
    targets = tokenizer(example['target'], truncation=True, padding='max_length', max_length=max_target_length)
    inputs['labels'] = targets['input_ids']
    return inputs

tokenized_data = dataset.map(preprocess, batched=True, remove_columns=dataset['train'].column_names)

Map: 100%|██████████| 2688/2688 [00:00<00:00, 4093.28 examples/s]


In [4]:
# Training configuration
args = Seq2SeqTrainingArguments(
    output_dir='t5_intent_response_model',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir='./logs',
    logging_steps=50,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

# Evaluation metric
rouge = evaluate.load('rouge')

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    return rouge.compute(predictions=decoded_preds, references=decoded_labels)

In [5]:
# Train the model
trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

c:\Users\Asus\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 24184
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 9069
  Number of trainable parameters = 60506624
  1%|          | 50/9069 [00:16<46:33,  3.23it/s] 

{'loss': 5.7627, 'learning_rate': 1.9889734259565556e-05, 'epoch': 0.02}


  1%|          | 100/9069 [00:32<46:39,  3.20it/s]

{'loss': 4.4583, 'learning_rate': 1.977946851913111e-05, 'epoch': 0.03}


  2%|▏         | 150/9069 [00:47<45:48,  3.24it/s]

...

Model weights saved in t5_intent_response_model\checkpoint-9000\pytorch_model.bin
tokenizer config file saved in t5_intent_response_model\checkpoint-9000\tokenizer_config.json
Special tokens file saved in t5_intent_response_model\checkpoint-9000\special_tokens_map.json
Deleting older checkpoint [t5_intent_response_model\checkpoint-8000] due to args.save_total_limit
100%|█████████▉| 9050/9069 [53:42<00:05,  3.19it/s]

{'loss': 1.4275, 'learning_rate': 4.190098136508987e-08, 'epoch': 2.99}


100%|██████████| 9069/9069 [53:48<00:00,  2.93it/s]***** Running Evaluation *****
  Num examples = 2688
  Batch size = 8
                                                   
100%|██████████| 9069/9069 [57:06<00:00,  2.93it/s]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 9069/9069 [57:06<00:00,  2.65it/s]

{'eval_loss': 1.1699498891830444, 'eval_rouge1': 0.19446302791821615, 'eval_rouge2': 0.10728213799300332, 'eval_rougeL': 0.17024564120189278, 'eval_rougeLsum': 0.17014773440694453, 'eval_runtime': 198.122, 'eval_samples_per_second': 13.567, 'eval_steps_per_second': 1.696, 'epoch': 3.0}
{'train_runtime': 3426.7818, 'train_samples_per_second': 21.172, 'train_steps_per_second': 2.647, 'train_loss': 1.6643453659644818, 'epoch': 3.0}


TrainOutput(global_step=9069, training_loss=1.6643453659644818, metrics={'train_runtime': 3426.7818, 'train_samples_per_second': 21.172, 'train_steps_per_second': 2.647, 'train_loss': 1.6643453659644818, 'epoch': 3.0})